# VAE LSTM

In [2]:
# !pip install tensorflow


In [3]:
import pandas as pd
import numpy as np
data=pd.read_csv(r"C:\Users\dwill\OneDrive\Desktop\INTERN WORKS\wind_data.csv")

In [4]:
print(data.head())
print(data.shape)

   TurbID  Day Tmstamp  Wspd  Wdir   Etmp   Itmp   Ndir  Pab1  Pab2  Pab3  \
0       1    1   00:10  6.17 -3.99  30.73  41.80  25.92   1.0   1.0   1.0   
1       1    1   00:20  6.27 -2.18  30.60  41.63  20.91   1.0   1.0   1.0   
2       1    1   00:30  6.42 -0.73  30.52  41.52  20.91   1.0   1.0   1.0   
3       1    1   00:40  6.25  0.89  30.49  41.38  20.91   1.0   1.0   1.0   
4       1    1   00:50  6.10 -1.03  30.47  41.22  20.91   1.0   1.0   1.0   

   Prtv    Patv  
0 -0.25  494.66  
1 -0.24  509.76  
2 -0.26  542.53  
3 -0.23  509.36  
4 -0.27  482.21  
(10000, 13)


In [5]:
data.isnull().sum()

TurbID     0
Day        0
Tmstamp    0
Wspd       0
Wdir       0
Etmp       0
Itmp       0
Ndir       0
Pab1       0
Pab2       0
Pab3       0
Prtv       0
Patv       0
dtype: int64

In [6]:
data=data.dropna()
data.isnull().sum()

TurbID     0
Day        0
Tmstamp    0
Wspd       0
Wdir       0
Etmp       0
Itmp       0
Ndir       0
Pab1       0
Pab2       0
Pab3       0
Prtv       0
Patv       0
dtype: int64

In [7]:
data.shape

(10000, 13)

In [8]:
data[['hour', 'minute']] = data['Tmstamp'].str.split(':', expand=True)
data['hour'] = pd.to_numeric(data['hour'])
data['minute'] = pd.to_numeric(data['minute'])
data.drop(columns=['Tmstamp'], inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   TurbID  10000 non-null  int64  
 1   Day     10000 non-null  int64  
 2   Wspd    10000 non-null  float64
 3   Wdir    10000 non-null  float64
 4   Etmp    10000 non-null  float64
 5   Itmp    10000 non-null  float64
 6   Ndir    10000 non-null  float64
 7   Pab1    10000 non-null  float64
 8   Pab2    10000 non-null  float64
 9   Pab3    10000 non-null  float64
 10  Prtv    10000 non-null  float64
 11  Patv    10000 non-null  float64
 12  hour    10000 non-null  int64  
 13  minute  10000 non-null  int64  
dtypes: float64(10), int64(4)
memory usage: 1.1 MB


In [9]:
desired_order = [
    'TurbID', 'Day', 'hour', 'minute','Wspd', 'Wdir', 'Etmp', 'Itmp', 'Ndir',
    'Pab1', 'Pab2', 'Pab3', 'Prtv', 'Patv']
data = data.reindex(columns=desired_order)
print(data)

      TurbID  Day  hour  minute  Wspd  Wdir   Etmp   Itmp    Ndir  Pab1  Pab2  \
0          1    1     0      10  6.17 -3.99  30.73  41.80   25.92  1.00  1.00   
1          1    1     0      20  6.27 -2.18  30.60  41.63   20.91  1.00  1.00   
2          1    1     0      30  6.42 -0.73  30.52  41.52   20.91  1.00  1.00   
3          1    1     0      40  6.25  0.89  30.49  41.38   20.91  1.00  1.00   
4          1    1     0      50  6.10 -1.03  30.47  41.22   20.91  1.00  1.00   
...      ...  ...   ...     ...   ...   ...    ...    ...     ...   ...   ...   
9995       1   72     2      10  6.72  4.66  32.53  43.40  404.58  1.00  1.00   
9996       1   72     2      20  4.25 -2.10  32.33  42.71  413.38  0.99  0.99   
9997       1   72     2      30  3.50  3.53  32.26  42.46  411.41  0.99  0.99   
9998       1   72     2      40  5.39  4.66  32.11  42.31  425.11  0.99  0.99   
9999       1   72     2      50  6.23 -0.43  31.93  42.25  444.62  1.00  1.00   

      Pab3  Prtv    Patv  


In [10]:
import pandas as pd
import numpy as np
def drop_outliers_iqr(dataframe):
    q1 = dataframe.quantile(0.25)
    q3 = dataframe.quantile(0.75)
    iqr = q3 - q1
    #1.5 is threshold
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    outliers = ((dataframe < lower_bound) | (dataframe > upper_bound)).any(axis=1)
    cleaned_dataframe = dataframe[~outliers]
    return cleaned_dataframe
cleaned_df = drop_outliers_iqr(data)
print("Original DataFrame:")
print(data)
print("\nDataFrame after dropping outliers:")
print(cleaned_df)

Original DataFrame:
      TurbID  Day  hour  minute  Wspd  Wdir   Etmp   Itmp    Ndir  Pab1  Pab2  \
0          1    1     0      10  6.17 -3.99  30.73  41.80   25.92  1.00  1.00   
1          1    1     0      20  6.27 -2.18  30.60  41.63   20.91  1.00  1.00   
2          1    1     0      30  6.42 -0.73  30.52  41.52   20.91  1.00  1.00   
3          1    1     0      40  6.25  0.89  30.49  41.38   20.91  1.00  1.00   
4          1    1     0      50  6.10 -1.03  30.47  41.22   20.91  1.00  1.00   
...      ...  ...   ...     ...   ...   ...    ...    ...     ...   ...   ...   
9995       1   72     2      10  6.72  4.66  32.53  43.40  404.58  1.00  1.00   
9996       1   72     2      20  4.25 -2.10  32.33  42.71  413.38  0.99  0.99   
9997       1   72     2      30  3.50  3.53  32.26  42.46  411.41  0.99  0.99   
9998       1   72     2      40  5.39  4.66  32.11  42.31  425.11  0.99  0.99   
9999       1   72     2      50  6.23 -0.43  31.93  42.25  444.62  1.00  1.00   

      P

In [11]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
cleaned_df_normalized = scaler.fit_transform(cleaned_df)
cleaned_df_normalized_df = pd.DataFrame(cleaned_df_normalized, columns=cleaned_df.columns)
print(cleaned_df_normalized_df)

      TurbID  Day      hour  minute      Wspd      Wdir      Etmp      Itmp  \
0        0.0  0.0  0.000000     0.2  0.364055  0.310772  0.611111  0.625951   
1        0.0  0.0  0.000000     0.4  0.370639  0.397045  0.606421  0.619790   
2        0.0  0.0  0.000000     0.6  0.380513  0.466158  0.603535  0.615803   
3        0.0  0.0  0.000000     0.8  0.369322  0.543375  0.602453  0.610729   
4        0.0  0.0  0.000000     1.0  0.359447  0.451859  0.601732  0.604929   
...      ...  ...       ...     ...       ...       ...       ...       ...   
6946     0.0  1.0  0.086957     0.2  0.400263  0.723070  0.676046  0.683943   
6947     0.0  1.0  0.086957     0.4  0.237656  0.400858  0.668831  0.658934   
6948     0.0  1.0  0.086957     0.6  0.188282  0.669209  0.666306  0.649873   
6949     0.0  1.0  0.086957     0.8  0.312706  0.723070  0.660895  0.644436   
6950     0.0  1.0  0.086957     1.0  0.368005  0.480458  0.654401  0.642262   

          Ndir      Pab1      Pab2      Pab3      P

In [12]:
x=cleaned_df_normalized_df.drop(columns=['Patv'])
y=cleaned_df_normalized_df["Patv"]

In [13]:
# wrapper method
# Recursive Feature Elimination (RFE)
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
model = LinearRegression()
rfe = RFE(model, n_features_to_select=6) 
rfe = rfe.fit(x, y)
# Get selected features
selected_features = x.columns[rfe.support_]
df_reduced = x[selected_features]
print(df_reduced)

          Wspd      Etmp      Itmp      Pab1      Pab2      Pab3
0     0.364055  0.611111  0.625951  0.010989  0.015385  0.010989
1     0.370639  0.606421  0.619790  0.010989  0.015385  0.010989
2     0.380513  0.603535  0.615803  0.010989  0.015385  0.010989
3     0.369322  0.602453  0.610729  0.010989  0.015385  0.010989
4     0.359447  0.601732  0.604929  0.010989  0.015385  0.010989
...        ...       ...       ...       ...       ...       ...
6946  0.400263  0.676046  0.683943  0.010989  0.015385  0.010989
6947  0.237656  0.668831  0.658934  0.000000  0.000000  0.000000
6948  0.188282  0.666306  0.649873  0.000000  0.000000  0.000000
6949  0.312706  0.660895  0.644436  0.000000  0.000000  0.000000
6950  0.368005  0.654401  0.642262  0.010989  0.015385  0.010989

[6951 rows x 6 columns]


In [14]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df_reduced,y,test_size=0.1,random_state=42)
x_train.shape,y_test.shape

((6255, 6), (696,))

In [15]:
# !pip show tensorflow

In [16]:
# !pip uninstall tensorflow -y

In [17]:
# !pip install tensorflow==2.15

In [18]:
import tensorflow as tf
from tensorflow.keras import layers, models 
import numpy as np
latent_dim = 6
input_shape = 6
conv_input_shape = (input_shape, 1)

# Encoder
encoder_inputs = layers.Input(shape=conv_input_shape)
x = layers.Conv1D(128, kernel_size=3, activation='elu', padding='same')(encoder_inputs)
x = layers.Conv1D(64, kernel_size=3, activation='elu', padding='same')(x)
x = layers.Conv1D(32, kernel_size=3, activation='elu', padding='same')(x)
x = layers.Flatten()(x)
x = layers.Dense(256, activation=tf.nn.leaky_relu)(x)  # Leaky ReLU activation
x = layers.Dense(128, activation=tf.nn.leaky_relu)(x)  # Leaky ReLU activation
x = layers.Dense(64, activation=tf.nn.leaky_relu)(x)  # Leaky ReLU activation
z_mean = layers.Dense(latent_dim)(x)
z_log_var = layers.Dense(latent_dim)(x)

# Sampler
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.random.normal(shape=tf.shape(z_mean))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon
z = layers.Lambda(sampling)([z_mean, z_log_var])

# Decoder
decoder_inputs = layers.Input(shape=(latent_dim,))
x = layers.Dense(64, activation=tf.nn.leaky_relu)(decoder_inputs)  # Leaky ReLU activation
x = layers.Dense(128, activation=tf.nn.leaky_relu)(x)  # Leaky ReLU activation
x = layers.Dense(256, activation=tf.nn.leaky_relu)(x)  # Leaky ReLU activation
x = layers.Dense(input_shape * 32, activation=tf.nn.leaky_relu)(x)  # To match the flattened Conv1D output
x = layers.Reshape((input_shape, 32))(x)
x = layers.Conv1D(32, kernel_size=3, activation='elu', padding='same')(x)
x = layers.Conv1D(64, kernel_size=3, activation='elu', padding='same')(x)
x = layers.Conv1D(128, kernel_size=3, activation='elu', padding='same')(x)
decoder_outputs = layers.Conv1D(1, kernel_size=3, activation='sigmoid', padding='same')(x)
decoder_outputs = layers.Flatten()(decoder_outputs)

encoder = models.Model(encoder_inputs, [z_mean, z_log_var, z], name='encoder')
decoder = models.Model(decoder_inputs, decoder_outputs, name='decoder')

outputs = decoder(encoder(encoder_inputs)[2])
vae = models.Model(encoder_inputs, outputs, name='vae')

# Loss
def vae_loss(x, outputs):
    x_decoded_mean = outputs
    z_log_var = outputs[1]
    z_mean = outputs[0]
    
    reconstruction_loss = tf.reduce_mean(tf.keras.losses.mean_squared_error(x, x_decoded_mean), axis=-1)
    kl_loss = -0.5 * tf.reduce_mean(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=-1)
    return tf.reduce_mean(reconstruction_loss + kl_loss)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

vae.compile(optimizer=optimizer, loss=vae_loss)

vae.fit(x_train, x_train, epochs=200, batch_size=64, validation_data=(x_test, x_test))

latent_train = encoder.predict(x_train)[2]  # Extracting only the z vector
reconstructed_data = decoder.predict(latent_train)
latent_train_array = np.array(latent_train)
sequence_length = latent_train_array.shape[1]
print("Sequence Length:", sequence_length)



Epoch 1/200

98/98 [==============================] - 15s 42ms/step - loss: 0.0913 - val_loss: 0.0803
Epoch 2/200
98/98 [==============================] - 3s 29ms/step - loss: 0.0765 - val_loss: 0.0772
Epoch 3/200
98/98 [==============================] - 3s 29ms/step - loss: 0.0738 - val_loss: 0.0738
Epoch 4/200
98/98 [==============================] - 3s 28ms/step - loss: 0.0698 - val_loss: 0.0710
Epoch 5/200
98/98 [==============================] - 3s 31ms/step - loss: 0.0722 - val_loss: 0.0729
Epoch 6/200
98/98 [==============================] - 3s 30ms/step - loss: 0.0723 - val_loss: 0.0721
Epoch 7/200
98/98 [==============================] - 3s 30ms/step - loss: 0.0669 - val_loss: 0.0759
Epoch 8/200
98/98 [==============================] - 3s 31ms/step - loss: 0.0684 - val_loss: 0.0716
Epoch 9/200
98/98 [==============================] - 3s 31ms/step - loss: 0.0690 - val_loss: 0.0704
Epoch 10/200
98/98 [==============================] - 3s 30ms/step - loss: 0.0672 - val_loss: 0.

In [19]:
# !pip install numpy==1.25.2

   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.5 MB 162.5 kB/s eta 0:01:36
   ---------------------------------------- 0.0/15.5 MB 245.8 kB/s eta 0:01:04
   ---------------------------------------- 0.1/15.5 MB 490.2 kB/s eta 0:00:32
    --------------------------------------- 0.4/15.5 MB 1.5 MB/s eta 0:00:10
   --- ------------------------------------ 1.2/15.5 MB 4.4 MB/s eta 0:00:04
   ------ --------------------------------- 2.6/15.5 MB 8.8 MB/s eta 0:00:02
   ------ --------------------------------- 2.6/15.5 MB 7.0 MB/s eta 0:00:02
   --------- ------------------------------ 3.7/15.5 MB 9.1 MB/s eta 0:00:02
   ---------- ----------------------------- 4.1/15.5 MB 8.9 MB/s eta 0:00:02
   ---------- -------

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jax 0.7.0 requires ml_dtypes>=0.5.0, but you have ml-dtypes 0.2.0 which is incompatible.
jax 0.7.0 requires numpy>=1.26, but you have numpy 1.25.2 which is incompatible.
jaxlib 0.7.0 requires ml_dtypes>=0.5.0, but you have ml-dtypes 0.2.0 which is incompatible.
jaxlib 0.7.0 requires numpy>=1.26, but you have numpy 1.25.2 which is incompatible.
scikit-criteria 0.9 requires numpy>=2.0, but you have numpy 1.25.2 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
# reconstruction loss
reconstructed_test = vae.predict(x_test)
reconstruction_errors_test = np.mean(np.square(x_test - reconstructed_test), axis=1)
print(f"Mean Reconstruction Error for test data: {np.mean(reconstruction_errors_test):.4f}")
reconstructed_train = vae.predict(x_train)
reconstruction_errors_train = np.mean(np.square(x_train - reconstructed_train), axis=1)
print(f"Mean Reconstruction Error for train data: {np.mean(reconstruction_errors_train):.4f}")

22/22 [==============================] - 1s 7ms/step
Mean Reconstruction Error for test data: 0.0261
196/196 [==============================] - 2s 9ms/step
Mean Reconstruction Error for train data: 0.0256


In [20]:
import numpy as np
from sklearn.metrics import mean_squared_error
original_data_array = x_train.values
original_data_flat = original_data_array.flatten()
reconstructed_data_flat = reconstructed_data.flatten()
mse = mean_squared_error(original_data_flat, reconstructed_data_flat)
print("Mean Squared Error (MSE):", mse)
rmse = np.sqrt(mse)
print("Root Mean Squared Error (RMSE):", rmse)

Mean Squared Error (MSE): 0.0256325597745432
Root Mean Squared Error (RMSE): 0.16010171696313316


In [21]:
latent_train_reshaped = latent_train_array.reshape(latent_train_array.shape[0], latent_train_array.shape[1],1)
print(latent_train_reshaped.shape)

(6255, 6, 1)


In [22]:
latent_test = encoder.predict(x_test)[2]
latent_test_array = np.array(latent_test)
latent_test_reshaped = latent_test_array.reshape(latent_test_array.shape[0], latent_test_array.shape[1],1)
print(latent_test_reshaped.shape)

22/22 [==============================] - 0s 8ms/step
(696, 6, 1)


In [23]:
from sklearn.model_selection import KFold
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Prepare data
latent_data = np.concatenate((latent_train_reshaped, latent_test_reshaped), axis=0)
target_data = np.concatenate((y_train, y_test), axis=0)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

fold = 1
for train_index, test_index in kf.split(latent_data):
    print(f"Training on Fold {fold}...")
    
    x_train_fold, x_test_fold = latent_data[train_index], latent_data[test_index]
    y_train_fold, y_test_fold = target_data[train_index], target_data[test_index]
    
    # Define LSTM model
    vae_lstm = Sequential()
    vae_lstm.add(LSTM(units=150, activation='relu', return_sequences=True, input_shape=(6, 1)))
    vae_lstm.add(Dropout(0.2))
    vae_lstm.add(LSTM(units=150, activation='relu'))
    vae_lstm.add(Dense(units=1))
    
    optimizer = Adam(learning_rate=0.0001)
    vae_lstm.compile(optimizer=optimizer, loss='mean_squared_error')
    
    # Train model
    vae_lstm.fit(x_train_fold, y_train_fold, epochs=10, batch_size=8, verbose=1)
    
    # Evaluate model
    loss = vae_lstm.evaluate(x_test_fold, y_test_fold)
    print(f"Fold {fold} - Test Loss: {loss}")
    
    # Predictions and metrics
    y_pred = vae_lstm.predict(x_test_fold)
    mse = mean_squared_error(y_test_fold, y_pred)
    mae = mean_absolute_error(y_test_fold, y_pred)
    r2 = r2_score(y_test_fold, y_pred)
    
    print(f"Fold {fold} - Mean Squared Error (MSE):", mse)
    print(f"Fold {fold} - Mean Absolute Error (MAE):", mae)
    print(f"Fold {fold} - R-squared (R2) Score:", r2)
    
    fold += 1

Training on Fold 1...
Epoch 1/10
695/695 [==============================] - 20s 19ms/step - loss: 0.0982
Epoch 2/10
695/695 [==============================] - 13s 19ms/step - loss: 0.0276
Epoch 3/10
695/695 [==============================] - 13s 19ms/step - loss: 0.0101
Epoch 4/10
695/695 [==============================] - 13s 19ms/step - loss: 0.0067
Epoch 5/10
695/695 [==============================] - 14s 20ms/step - loss: 0.0050
Epoch 6/10
695/695 [==============================] - 13s 19ms/step - loss: 0.0043
Epoch 7/10
695/695 [==============================] - 13s 19ms/step - loss: 0.0038
Epoch 8/10
695/695 [==============================] - 14s 19ms/step - loss: 0.0034
Epoch 9/10
695/695 [==============================] - 14s 20ms/step - loss: 0.0032
Epoch 10/10
44/44 [==============================] - 2s 17ms/step - loss: 0.0018
Fold 1 - Test Loss: 0.001774831092916429
44/44 [==============================] - 2s 16ms/step
Fold 1 - Mean Squared Error (MSE): 0.001774830977327345

In [24]:
vae_lstm.save('vae_lstm.keras')
# from tensorflow.keras.models import load_model
# vae_lstm = load_model('vae_lstm.h5')

# LSTM

In [26]:
x_train = x_train.values
y_train = y_train.values
feature=1
x_train = x_train.reshape((x_train.shape[0],x_train.shape[1],feature))
print(x_train.shape)
x_test = x_test.values
y_test = y_test.values
x_test = x_test.reshape((x_test.shape[0],x_test.shape[1],feature))

(6255, 6, 1)


In [27]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
lstm = Sequential()
lstm.add(LSTM(units=150, activation='relu', return_sequences=True, input_shape=(6,1)))
lstm.add(Dropout(0.2))
lstm.add(LSTM(units=150, activation='relu'))
lstm.add(Dense(units=1))
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
lstm.compile(optimizer=optimizer, loss='mean_squared_error')
lstm.fit(x_train, y_train, epochs=10, batch_size=8)
loss = lstm.evaluate(x_test, y_test)
print("Test Loss:", loss)
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
y_pred = lstm.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error (MSE):", mse)
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error (MAE):", mae)
r2 = r2_score(y_test, y_pred)
print("R-squared (R2) Score:", r2)

Epoch 1/10
782/782 [==============================] - 25s 22ms/step - loss: 0.0884
Epoch 2/10
782/782 [==============================] - 18s 22ms/step - loss: 0.0317
Epoch 3/10
782/782 [==============================] - 17s 22ms/step - loss: 0.0113
Epoch 4/10
782/782 [==============================] - 17s 22ms/step - loss: 0.0081
Epoch 5/10
782/782 [==============================] - 17s 22ms/step - loss: 0.0068
Epoch 6/10
782/782 [==============================] - 17s 22ms/step - loss: 0.0058
Epoch 7/10
782/782 [==============================] - 17s 22ms/step - loss: 0.0052
Epoch 8/10
782/782 [==============================] - 17s 22ms/step - loss: 0.0046
Epoch 9/10
782/782 [==============================] - 17s 22ms/step - loss: 0.0041
Epoch 10/10
22/22 [==============================] - 1s 16ms/step - loss: 0.0017
Test Loss: 0.0016879051690921187
22/22 [==============================] - 1s 16ms/step
Mean Squared Error (MSE): 0.0016879051589465792
Mean Absolute Error (MAE): 0.02793461

# TOPSIS METHOD

In [30]:
import tensorflow as tf
print(tf.__version__)

!pip list

2.15.0
Package                          Version
-------------------------------- ------------
absl-py                          2.3.1
aiohttp                          3.9.5
aiosignal                        1.2.0
anyio                            4.0.0
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
arrow                            1.3.0
asgiref                          3.8.1
asttokens                        2.4.0
astunparse                       1.6.3
async-lru                        2.0.4
async-timeout                    4.0.2
attrs                            25.3.0
Babel                            2.13.0
backcall                         0.2.0
base58                           2.1.1
beautifulsoup4                   4.12.2
bitarray                         2.9.2
bleach                           6.1.0
cachetools                       6.2.0
certifi                          2021.10.8
cffi                             1.16.0
charset-normalizer               2.0.1


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
# !pip install scikit-criteria==0.7

In [10]:
# !pip install --upgrade scikit-criteria

  Using cached scikit_criteria-0.9-py3-none-any.whl.metadata (7.2 kB)
Using cached scikit_criteria-0.9-py3-none-any.whl (176 kB)
  Attempting uninstall: scikit-criteria
    Found existing installation: scikit-criteria 0.7
    Uninstalling scikit-criteria-0.7:
      Successfully uninstalled scikit-criteria-0.7


In [11]:
# import skcriteria
# print(skcriteria.__version__)

('0', '9')


In [32]:
# !pip install scikit-criteria
!pip show skcriteria

In [32]:
!pip install --upgrade pip

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
    --------------------------------------- 0.0/1.8 MB 435.7 kB/s eta 0:00:04
   -- ------------------------------------- 0.1/1.8 MB 871.5 kB/s eta 0:00:02
   ---------- ----------------------------- 0.5/1.8 MB 3.3 MB/s eta 0:00:01
   ------------------------ --------------- 1.1/1.8 MB 6.1 MB/s eta 0:00:01
   ---------------------------------------  1.8/1.8 MB 7.9 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 7.4 MB/s eta 0:00:00


ERROR: To modify pip, please run the following command:
C:\Users\dwill\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Final

In [45]:
# updated_pipeline.py
import pandas as pd
import numpy as np

# ===========================
# Load & preprocess dataset
# ===========================
data = pd.read_csv(r"C:\Users\dwill\OneDrive\Desktop\INTERN WORKS\wind_data.csv")
data = data.dropna()

# Split Tmstamp
data[['hour', 'minute']] = data['Tmstamp'].str.split(':', expand=True)
data['hour'] = pd.to_numeric(data['hour'])
data['minute'] = pd.to_numeric(data['minute'])
data.drop(columns=['Tmstamp'], inplace=True)

desired_order = [
    'TurbID', 'Day', 'hour', 'minute','Wspd', 'Wdir', 'Etmp', 'Itmp', 'Ndir',
    'Pab1', 'Pab2', 'Pab3', 'Prtv', 'Patv'
]
data = data.reindex(columns=desired_order)

# Outlier removal with IQR
def drop_outliers_iqr(df):
    q1 = df.quantile(0.25)
    q3 = df.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    mask = ((df < lower) | (df > upper)).any(axis=1)
    return df[~mask]

cleaned_df = drop_outliers_iqr(data)

# Normalization
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
cleaned_df_normalized = scaler.fit_transform(cleaned_df)
cleaned_df_normalized_df = pd.DataFrame(cleaned_df_normalized, columns=cleaned_df.columns)

x = cleaned_df_normalized_df.drop(columns=['Patv'])
y = cleaned_df_normalized_df['Patv'].to_numpy()  # make numpy array for later

# ===========================
# Manual TOPSIS Feature Ranking (feature-level)
# ===========================
# We treat each feature (column) as an alternative by transposing the matrix.
decision_matrix = x.values.T.astype(float)   # shape: (n_features, n_samples)
eps = 1e-12  # small value to avoid divide-by-zero

# Normalize across samples for each feature (vector normalization)
denom = np.sqrt((decision_matrix ** 2).sum(axis=1, keepdims=True))
denom[denom == 0] = eps
norm_matrix = decision_matrix / denom  # shape: (n_features, n_samples)

# Use equal weights for criteria (samples) when treating features as alternatives
# (This is a heuristic; you can change weights if desired)
weights = np.ones(norm_matrix.shape[1])  # one weight per sample/criterion
weighted_matrix = norm_matrix * weights[np.newaxis, :]

# Ideal best and ideal worst (for each criterion/sample)
ideal_best = weighted_matrix.max(axis=0)
ideal_worst = weighted_matrix.min(axis=0)

# Distances of each feature (alternative) to ideal best/worst
dist_best = np.sqrt(((weighted_matrix - ideal_best[np.newaxis, :]) ** 2).sum(axis=1))
dist_worst = np.sqrt(((weighted_matrix - ideal_worst[np.newaxis, :]) ** 2).sum(axis=1))

# Closeness coefficient per feature
closeness = dist_worst / (dist_best + dist_worst + eps)

feature_scores = pd.DataFrame({
    "Feature": x.columns,
    "Closeness": closeness
})
feature_scores["Rank"] = feature_scores["Closeness"].rank(ascending=False, method='min')
feature_scores = feature_scores.sort_values(by="Rank").reset_index(drop=True)

print("\nTOPSIS Feature Ranking:")
print(feature_scores)

# Select top 6 features
top_features = feature_scores.head(6)["Feature"].tolist()
df_reduced = x[top_features].copy()
print("\nSelected Features via TOPSIS:")
print(df_reduced.head())

# ===========================
# Train/Test Split
# ===========================
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_reduced, y, test_size=0.1, random_state=42)

# Convert y to numpy arrays (if not already)
y_train = np.asarray(y_train).reshape(-1)
y_test = np.asarray(y_test).reshape(-1)

# ===========================
# VAE Model (fixed shapes + KL via add_loss)
# ===========================
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

latent_dim = 6
n_features = x_train.shape[1]  # should be 6

# VAE input shape: (timesteps/features, channels=1)
encoder_inputs = layers.Input(shape=(n_features, 1), name="encoder_input")

# Encoder
e = layers.Conv1D(64, kernel_size=3, activation='elu', padding='same')(encoder_inputs)
e = layers.Conv1D(32, kernel_size=3, activation='elu', padding='same')(e)
e = layers.Flatten()(e)
e = layers.Dense(128, activation='relu')(e)

z_mean = layers.Dense(latent_dim, name='z_mean')(e)
z_log_var = layers.Dense(latent_dim, name='z_log_var')(e)

def sampling_layer(args):
    zm, zv = args
    eps = tf.random.normal(shape=tf.shape(zm))
    return zm + tf.exp(0.5 * zv) * eps

z = layers.Lambda(sampling_layer, name='z')([z_mean, z_log_var])

encoder = models.Model(encoder_inputs, [z_mean, z_log_var, z], name='encoder')

# Decoder (produce same shape as input: (n_features, 1))
decoder_inputs = layers.Input(shape=(latent_dim,), name="decoder_input")
d = layers.Dense(128, activation='relu')(decoder_inputs)
d = layers.Dense(n_features * 16, activation='relu')(d)
d = layers.Reshape((n_features, 16))(d)
d = layers.Conv1D(16, kernel_size=3, activation='elu', padding='same')(d)
decoder_outputs = layers.Conv1D(1, kernel_size=3, activation='sigmoid', padding='same', name='decoder_output')(d)

decoder = models.Model(decoder_inputs, decoder_outputs, name='decoder')

# VAE model: encoder -> decoder(z)
encoded = encoder(encoder_inputs)[2]
reconstructed = decoder(encoded)
vae = models.Model(encoder_inputs, reconstructed, name='vae')

# Add KL loss via add_loss (works with Keras symbolic tensors)
kl_loss = -0.5 * tf.reduce_mean(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
vae.add_loss(kl_loss)

# Compile with reconstruction loss only (KL is added already)
vae.compile(optimizer=Adam(1e-3), loss='mse')

# Prepare inputs: expand dims to add channel dimension
X_train_vae = np.expand_dims(x_train.values.astype(np.float32), axis=-1)  # shape (samples, n_features, 1)
X_test_vae = np.expand_dims(x_test.values.astype(np.float32), axis=-1)

# Fit VAE
vae.fit(X_train_vae, X_train_vae,
        validation_data=(X_test_vae, X_test_vae),
        epochs=50, batch_size=64)

# ===========================
# Get latent representations
# ===========================
latent_train = encoder.predict(X_train_vae)[2]  # shape (n_samples, latent_dim)
latent_test = encoder.predict(X_test_vae)[2]

# reshape for LSTM input: (samples, timesteps=latent_dim, features=1)
latent_train_reshaped = latent_train.reshape(latent_train.shape[0], latent_train.shape[1], 1)
latent_test_reshaped = latent_test.reshape(latent_test.shape[0], latent_test.shape[1], 1)

# ===========================
# LSTM on Latent Features with KFold
# ===========================
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

latent_data = np.concatenate((latent_train_reshaped, latent_test_reshaped), axis=0)
target_data = np.concatenate((y_train, y_test), axis=0)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

fold = 1
for train_idx, test_idx in kf.split(latent_data):
    print(f"\nTraining on Fold {fold}...")
    X_tr, X_val = latent_data[train_idx], latent_data[test_idx]
    Y_tr, Y_val = target_data[train_idx], target_data[test_idx]

    model = Sequential([
        LSTM(128, activation='tanh', return_sequences=True, input_shape=(latent_dim, 1)),
        Dropout(0.2),
        LSTM(64, activation='tanh'),
        Dense(1)
    ])

    model.compile(optimizer=Adam(1e-4), loss='mse')
    model.fit(X_tr, Y_tr, epochs=10, batch_size=16, verbose=1)

    loss = model.evaluate(X_val, Y_val, verbose=0)
    print(f"Fold {fold} - Test Loss: {loss}")

    Y_pred = model.predict(X_val)
    print(f"Fold {fold} - MSE: {mean_squared_error(Y_val, Y_pred)}")
    print(f"Fold {fold} - MAE: {mean_absolute_error(Y_val, Y_pred)}")
    print(f"Fold {fold} - R2: {r2_score(Y_val, Y_pred)}")

    fold += 1

# Save final model (last fold model)
model.save('vae_lstm_update.h5')



TOPSIS Feature Ranking:
   Feature  Closeness  Rank
0     Wdir   0.457240   1.0
1     Itmp   0.450577   2.0
2     Prtv   0.450134   3.0
3     Etmp   0.449717   4.0
4     Ndir   0.444554   5.0
5     Wspd   0.438209   6.0
6      Day   0.435778   7.0
7     hour   0.428405   8.0
8   minute   0.422750   9.0
9     Pab1   0.417262  10.0
10    Pab3   0.417256  11.0
11    Pab2   0.398499  12.0
12  TurbID   0.000000  13.0

Selected Features via TOPSIS:
       Wdir      Itmp      Prtv      Etmp      Ndir      Wspd
0  0.310772  0.625951  0.500000  0.611111  0.271912  0.364055
1  0.397045  0.619790  0.541667  0.606421  0.266146  0.370639
2  0.466158  0.615803  0.458333  0.603535  0.266146  0.380513
3  0.543375  0.610729  0.583333  0.602453  0.266146  0.369322
4  0.451859  0.604929  0.416667  0.601732  0.266146  0.359447

Epoch 1/50
98/98 [==============================] - 9s 34ms/step - loss: 0.0323 - val_loss: 0.0303
Epoch 2/50
98/98 [==============================] - 1s 14ms/step - loss: 0.0316 

C:\Users\dwill\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
